In [121]:
from bs4 import BeautifulSoup
import requests
import time, os

import math
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [18]:
county_census_info = pd.read_pickle("../Data/county_census_info.pkl")

In [22]:
indices = county_census_info.index
print(indices[0])

('Autauga County', 'Alabama')


In [126]:
def isfloat(value_string_reduced):
  try:
    float(value_string_reduced)
    return float(value_string_reduced)
  except ValueError:
    return math.nan

def grabInitialMetric(soup, input_id):
    this_div = soup.find('div', id=input_id)
    this_span = this_div.find('span', id='initialMetric')
#     return isfloat(this_span.text)
    return isfloat(this_span.text.split(" ")[0])

def grabPerMetric(soup, input_id):
    this_div = soup.find('div', id=input_id)
    sub_div = this_div.find('div', class_='rates')
    per_text = sub_div.text
    line_of_interest = per_text.split("\n")[1].split(" ")    
    while('' in line_of_interest): 
        line_of_interest.remove('')
    return isfloat(line_of_interest[0].replace("(",''))
    

In [124]:
def grabCovidData(soup):

    county_dict = {}

#     top_soup = soup

#     time.sleep(1)

    svi_span = soup.find('span', id='svi_rank')
    svi_value = isfloat(svi_span.text)
    county_dict['SVI'] = svi_value
    
#     time.sleep(1)
    
    ccvi_span = soup.find('span', id='ccvi_score')
    ccvi_value = isfloat(ccvi_span.text)
    county_dict['CCVI'] = ccvi_value
    
#     cases_div = soup.find('div', id='cases-timeseries-wrapper')
#     cases_span = cases_div.find('span', id='initialMetric')
#     county_dict['Cases'] = isfloat(cases_span.text)
    
    county_dict['Cases'] = grabInitialMetric(soup, 'cases-timeseries-wrapper')
    county_dict['Cases per 100k'] = grabPerMetric(soup, 'cases-timeseries-wrapper')

    county_dict['Deaths'] = grabInitialMetric(soup, 'deaths-timeseries-wrapper')
    county_dict['Deaths per 100k'] = grabPerMetric(soup, 'deaths-timeseries-wrapper')

    county_dict['Percent Positivity'] = grabInitialMetric(soup, 'positivity-timeseries-wrapper')
    
    county_dict['Testing Volume'] = grabInitialMetric(soup, 'testing-timeseries-wrapper')
    county_dict['Testing Volume per 100k'] = grabPerMetric(soup, 'testing-timeseries-wrapper')

    county_dict['New Hospital Admissions'] = grabInitialMetric(soup, 'hospital-admissions-timeseries-wrapper')
    county_dict['New Hospital Admissions per 100 beds'] = grabPerMetric(soup, 'hospital-admissions-timeseries-wrapper')

    county_dict['Percent Beds Used (Covid)'] = grabInitialMetric(soup, 'hospital-percent-beds-timeseries-wrapper')
    
    
        
#     cases_rates_div = cases_div.find('div', class_='rates')
#     cases_per_100k_text = cases_rates_div.text
#     line_of_interest = cases_per_100k_text.split("\n")[1].split(" ")    
#     while('' in line_of_interest): 
#         line_of_interest.remove('')
#     county_dict['Cases per 100k'] = isfloat(line_of_interest[0].replace("(",''))

    
    
#     print(cases_per_100k_text.split("\n"))
#     print(cases_per_100k_text.split("\n")[0])
#     print(cases_per_100k_text.split("\n")[0].replace("(",''))
    
#     i = 0
#     for tbody in table_div.find_all('tbody'):
#         tbody_text = tbody.text

#         i+=1
#         if(i==1): continue
            
#         # read table in which returns a lot of text, then parse the text and drop various things to get the data
                
#         list_of_text = tbody_text.split("\n")
        
#         while('' in list_of_text): 
#             list_of_text.remove('')
#         while('\ue840\ue83f' in list_of_text):             
#             list_of_text.remove('\ue840\ue83f')
            
        
#         list_of_text.pop(0)    
        
#         # grab data titles and values and zip them together
        
#         for label, value in zip(list_of_text[0::2], list_of_text[1::2]):
#             value_number = removeStringCharacters(value)
#             county_dict[label] = value_number
    
#     return county_dict

    print(county_dict)

In [127]:
# open the cdc website and choose the state and county from the drop down menu

cdc_county_website = "https://covid.cdc.gov/covid-data-tracker/#county-view"

driver = webdriver.Chrome(chromedriver)
driver.get(cdc_county_website)

time.sleep(5)

select = Select(driver.find_element_by_id('list_select_state'))
select.select_by_visible_text('Texas')

time.sleep(1)

select = Select(driver.find_element_by_id('list_select_county'))
select.select_by_visible_text('Travis')

time.sleep(15)

# scrape data from the page

soup = BeautifulSoup(driver.page_source)

grabCovidData(soup)


driver.close()


{'SVI': 0.38, 'CCVI': 0.11, 'Cases': 4117.0, 'Cases per 100k': 323.17, 'Deaths': 27.0, 'Deaths per 100k': 2.12, 'Percent Positivity': 13.11, 'Testing Volume': 29397.0, 'Testing Volume per 100k': 2307.54, 'New Hospital Admissions': 438.0, 'New Hospital Admissions per 100 beds': 18.63, 'Percent Beds Used (Covid)': 18.93}
